In [ ]:
#!pip install pywin32
#!pip install pytz

In [ ]:
import pandas as pd
import pytz
import time
from win32com.client import Dispatch

In [ ]:
start_time = time.time()

outlook = Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)
sent_items = outlook.GetDefaultFolder(5)
folders = [inbox, sent_items]

data = []

def filter_mckinsey_email(emails):
    return [email for email in emails if '@mckinsey.com' in email]

for folder in folders:
    all_items = folder.Items

    for msg in all_items:
        if msg.Class == 43:
            if msg.SenderEmailType == 'EX':
                exchange_user = msg.Sender.GetExchangeUser()
                if exchange_user is not None:
                    sender_email = exchange_user.PrimarySmtpAddress
                else:
                    sender_email = "Exchange user not found"
            else:
                sender_email = msg.SenderEmailAddress

            email_date = msg.ReceivedTime
            email_date = email_date.replace(tzinfo=pytz.UTC)

            to_emails = []
            cc_emails = []

            for recipient in msg.Recipients:
                if recipient.Type == 1:  # To
                    if recipient.AddressEntry.Type == "EX":
                        exchange_user = recipient.AddressEntry.GetExchangeUser()
                        if exchange_user is not None:
                            to_emails.append(exchange_user.PrimarySmtpAddress)
                        else:
                            to_emails.append("Exchange user not found")
                    else:
                        to_emails.append(recipient.Address)
                elif recipient.Type == 2:  # CC
                    if recipient.AddressEntry.Type == "EX":
                        exchange_user = recipient.AddressEntry.GetExchangeUser()
                        if exchange_user is not None:
                            cc_emails.append(exchange_user.PrimarySmtpAddress)
                        else:
                            cc_emails.append("Exchange user not found")
                    else:
                        cc_emails.append(recipient.Address)

            to_emails = filter_mckinsey_email(to_emails)
            cc_emails = filter_mckinsey_email(cc_emails)

            if '@mckinsey.com' in sender_email:
                combined_emails = ';'.join([sender_email] + to_emails + cc_emails)
                data.append({"Email_Date": email_date, "Email_Subject": msg.Subject, "Combined_Emails": combined_emails})

df = pd.DataFrame(data)

df.to_csv("mck_filtered_email_data.csv", index=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time to run the code: {elapsed_time/60:.1f} minutes")

df.head()

In [ ]:
# Read the data from the CSV file
data = pd.read_csv("McK_filtered_email_data.csv")

# Extract email addresses from the column containing multiple email addresses separated by semicolons
email_addresses = data['Combined_Emails'].str.split(';', expand=True).stack().reset_index(drop=True)

# Remove duplicates and create a new DataFrame with unique email addresses
unique_email_addresses = email_addresses.drop_duplicates().reset_index(drop=True)

# Add a semicolon at the end of each email address
unique_email_addresses = unique_email_addresses + ';'

# Create a new DataFrame with the unique email addresses
unique_email_df = pd.DataFrame(unique_email_addresses, columns=["Unique_Email_Addresses"])

# Sort the unique_email_df DataFrame in alphabetical order
unique_email_df = unique_email_df.sort_values(by="Unique_Email_Addresses")

# Save the unique email addresses to a new CSV file
unique_email_df.to_csv("unique_email_addresses.csv", index=False)
unique_email_df.head()